# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [1]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [48]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws
from pyspark.sql.functions import expr

In [59]:
# Crear una sesión de Spark
spark = SparkSession.builder\
        .master("local")\
        .appName("ETL-metadata-sitios")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# GOOGLE MAPS

**Dataset:** metadata-sitios

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [5]:
# Ruta local donde se encuentran los archivos JSON
ruta_local = "C:/Users/Usuario/Desktop/Proyecto Final/metadata-sitios"

# Lista para almacenar los DataFrames de metadatos
metadata_df = []

# Iteración sobre los archivos JSON del 1 al 11
for i in range(1, 12):
    # Lectura del archivo JSON utilizando Spark
    archivo = spark.read.json(f"{ruta_local}/{i}.json")
    
    # Conversión de la columna "MISC" a tipo de datos "string"
    archivo = archivo.withColumn("MISC", col("MISC").cast("string"))
    
    # Agregar el DataFrame a la lista de metadatos
    metadata_df.append(archivo)

# DataFrame final para almacenar los datos combinados
df_final = metadata_df[0]

# Unión de los DataFrames de metadatos utilizando unionByName
for dataframe in metadata_df[1:]:
    df_final = df_final.unionByName(dataframe)

# Sobrescribir metadata_df con el DataFrame final combinado
metadata_df = df_final

In [6]:
# Mostrar el DataFrame
metadata_df.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|                MISC|             address|avg_rating|            category|         description|             gmap_id|               hours|          latitude|          longitude|                name|num_of_reviews|price|    relative_results|               state|                 url|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|{[Wheelchair acce...|Porter Pharmacy, ...|       4.9|          [Pharmacy]|                NULL|0x88f16e41928ff68...|[[Friday, 8AM–6PM...|           32

In [13]:
#Muestra el número de filas del DataFrame
metadata_df.count()

3025011

In [11]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
    # Construye expresiones de agregación para contar nulos en cada columna
    expresiones_agregacion = [sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns]

    # Aplica las expresiones de agregación al dataframe
    conteo_nulos_por_columna = dataframe.agg(*expresiones_agregacion)

    # Muestra el resultado
    conteo_nulos_por_columna.show()

In [12]:
# Llama a la función con tu dataframe
conteo_nulos(metadata_df)

+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|  MISC|address|avg_rating|category|description|gmap_id| hours|latitude|longitude|name|num_of_reviews|  price|relative_results| state|url|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|690834|  80511|         0|   17419|    2770722|      0|787405|       0|        0|  37|             0|2749808|          295058|746455|  0|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+



In [20]:
#Función para la obtención de los primeros datos en columnas especificas

def obtener_primer_dato(dataframe, columna):
    # Selecciona la columna especificada y extrae el primer dato
    primer_dato = dataframe.select(columna).first()[0]
    return primer_dato

In [21]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_misc = obtener_primer_dato(metadata_df, "MISC")
print("Primer dato en la columna MISC:", primer_dato_misc)

Primer dato en la columna MISC: {[Wheelchair accessible entrance], null, null, null, null, null, null, null, [Mask required, Staff required to disinfect surfaces between visits], null, null, null, [Quick visit], null, null, [In-store shopping, Same-day delivery]}


In [22]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_desc = obtener_primer_dato(metadata_df, "description")
print("Primer dato en la columna description:", primer_dato_desc)

Primer dato en la columna description: None


In [23]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_hours = obtener_primer_dato(metadata_df, "hours")
print("Primer dato en la columna hours:", primer_dato_hours)

Primer dato en la columna hours: [['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM'], ['Sunday', 'Closed'], ['Monday', '8AM–6PM'], ['Tuesday', '8AM–6PM'], ['Wednesday', '8AM–12PM'], ['Thursday', '8AM–6PM']]


In [24]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_price = obtener_primer_dato(metadata_df, "price")
print("Primer dato en la columna price:", primer_dato_price)

Primer dato en la columna price: None


In [25]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_rr = obtener_primer_dato(metadata_df, "relative_results")
print("Primer dato en la columna relative_results:", primer_dato_rr)

Primer dato en la columna relative_results: ['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x88f16c32716531c1:0x5f19bdaa5044e4fa', '0x88f16e6e3f4a21df:0xcf495da9bb4d89ea']


In [26]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_state = obtener_primer_dato(metadata_df, "state")
print("Primer dato en la columna state:", primer_dato_state)

Primer dato en la columna state: Open ⋅ Closes 6PM


In [27]:
# Lista de columnas a eliminar 
columnas_a_eliminar = ['MISC', 'state', 'price', 'hours', 'description', 'relative_results'] 

In [28]:
# Elimina las columnas especificadas 
df = metadata_df.select([columna for columna in metadata_df.columns if columna not in columnas_a_eliminar])

In [29]:
# Mostrar el DataFrame
df.show()

+--------------------+----------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+--------------------+
|             address|avg_rating|            category|             gmap_id|          latitude|          longitude|                name|num_of_reviews|                 url|
+--------------------+----------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+--------------------+
|Porter Pharmacy, ...|       4.9|          [Pharmacy]|0x88f16e41928ff68...|           32.3883|           -83.3571|     Porter Pharmacy|            16|https://www.googl...|
|City Textile, 300...|       4.5|  [Textile exporter]|0x80c2c98c0e3c16f...|        34.0188913|       -118.2152898|        City Textile|             6|https://www.googl...|
|San Soo Dang, 761...|       4.4| [Korean restaurant]|0x80c2c778e3b73d3...|        34.0580917|       -118.2921295|        San Soo Dang|     

In [31]:
# Encuentra las filas duplicadas en el DataFrame
duplicados = df.subtract(df.dropDuplicates())
# Muestra los registros duplicados
duplicados.show()

+-------+----------+--------+-------+--------+---------+----+--------------+---+
|address|avg_rating|category|gmap_id|latitude|longitude|name|num_of_reviews|url|
+-------+----------+--------+-------+--------+---------+----+--------------+---+
+-------+----------+--------+-------+--------+---------+----+--------------+---+



In [34]:
#Mostrar la estructura de mi dataframe
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- gmap_id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- num_of_reviews: long (nullable = true)
 |-- url: string (nullable = true)



In [35]:
# Elimina las 'gmap_id' duplicados 
df = df.dropDuplicates(['gmap_id'])

In [36]:
#Pasa la columna 'category' a tipo string 
df = df.withColumn("category", col("category").cast("string"))

In [37]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_category = obtener_primer_dato(df, "category")
print("Primer dato en la columna category:", primer_dato_category)

Primer dato en la columna category: [Bank, Car finance and loan company, Financial planner, Loan agency, Mortgage lender]


In [38]:
# Filtra las filas que contienen 'restaurant' en la columna 'category' 
df = df.filter((col("category").contains('restaurant')) | (col("category").contains('Restaurant')))

In [49]:
# Utiliza la función substring para eliminar el primer y último caracter de la columna 'category' 
df = df.withColumn("category", expr("substring(category, 2, length(category)-2)"))

In [40]:
# Llama a la función con tu dataframe y el nombre de la columna
primer_dato_address = obtener_primer_dato(df, "address")
print("Primer dato en la columna address:", primer_dato_address)

Primer dato en la columna address: Ceviche Jax, 27 Seminole Rd, Atlantic Beach, FL 32233, United States


In [43]:
# Separa la columna 'address' en 3 columnas: 'address', 'city' y 'state' 
split_col = split(df['address'], ',') 
df = df.withColumn('state', substring(split_col.getItem(3), 2, 2)) 
df = df.withColumn('city', split_col.getItem(2)) 
df = df.withColumn('address', concat_ws(',', split_col.getItem(0), split_col.getItem(1)))

In [44]:
# Mostrar el DataFrame
df.show()

+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+----------------+
|             address|avg_rating|            category|             gmap_id|          latitude|         longitude|                name|num_of_reviews|                 url|state|            city|
+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+----------------+
|La Posada Del Rey...|       4.8|[Mexican restaurant]|0x145c2bc56025f08...|        29.5051316|       -98.4581639|   La Posada Del Rey|             8|https://www.googl...|   TX|     San Antonio|
|Oasis Restaurant,...|       3.9|        [Restaurant]|0x145e82e3ab31e24...|29.571551999999997|      -104.3689397|    Oasis Restaurant|            98|https://www.googl...|   TX|        Presidio|
|Giovanni's Pizzer...|       3

In [45]:
# Filtrar las filas con estados 'NY', 'CA' 
estados_seleccionados = ['NY', 'CA'] 

df = df.filter(df['state'].isin(estados_seleccionados))

In [50]:
# Mostrar el DataFrame
df.show()

+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+--------------+
|             address|avg_rating|            category|             gmap_id|          latitude|         longitude|                name|num_of_reviews|                 url|state|          city|
+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+--------------+
|Char's Hot Dogs, ...|       4.7|Hot dog restauran...|0x153326668aff359...|        38.0461618|      -122.1616748|     Char's Hot Dogs|            38|https://www.googl...|   CA|       Benicia|
|New York Kim-Bob ...|       4.3|Korean restaurant...|0x4065fd476208a27...|40.764643199999995|-73.81192399999999|New York Kim-Bob ...|            68|https://www.googl...|   NY|        Queens|
|Rob's Lunch Box D...|       4.8|Restaur

## Carga de nuestro archivo

In [ ]:
# Ruta al archivo Parquet local 
file_path = 'PF_Google_yelp_Map/Notebook/metadata-sitios-limpio' 
# Escribe el DataFrame a un solo archivo CSV localmente
df.coalesce(1).write.csv(file_path, header=True, mode='overwrite')